In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define image size and batch size
IMG_SIZE = 224
BATCH_SIZE = 32

In [7]:
# Define data generators for train, validation and test sets
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/train',target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/test',target_size=(IMG_SIZE, IMG_SIZE),batch_size=BATCH_SIZE,class_mode='binary')

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [8]:
# Define the model
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [9]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)#training the data

Epoch 1/5
75/75 [==============================] - 333s 4s/step - loss: 0.5312 - accuracy: 0.7554 - val_loss: 0.3304 - val_accuracy: 0.8633
Epoch 2/5
75/75 [==============================] - 315s 4s/step - loss: 0.2464 - accuracy: 0.9029 - val_loss: 0.2238 - val_accuracy: 0.9033
Epoch 3/5
75/75 [==============================] - 309s 4s/step - loss: 0.1315 - accuracy: 0.9538 - val_loss: 0.0504 - val_accuracy: 0.9833
Epoch 4/5
75/75 [==============================] - 291s 4s/step - loss: 0.0530 - accuracy: 0.9858 - val_loss: 0.0220 - val_accuracy: 0.9967
Epoch 5/5
75/75 [==============================] - 315s 4s/step - loss: 0.0231 - accuracy: 0.9942 - val_loss: 0.0312 - val_accuracy: 0.9833


In [11]:
model.save("Model.h5","label.txt")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [12]:
#test your image
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

#load the model
model = load_model('/content/Model.h5')
#classes
class_names = ['You have Brain Tumor','You do not have Brain Tumor']
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [16]:
image = Image.open("/content/drive/MyDrive/Brain_Tumor_Detection/test/pred1.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

In [15]:
# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
confidence_score = prediction[0][index]
# Print prediction and confidence score
print("Result: ", class_names[index], end="")
print("\n")
print("Accuracy: ", confidence_score)

1/1 [==============================] - 0s 136ms/step
Result:  You have Brain Tumor

Accuracy:  1.0
